# Cuaderno Clase PLN - GloVe y FastText

**Objetivos:**
*   Conocer enfoques alternativos a Word2Vec: GloVe (conteos globales) y FastText (subpalabras).
*   Entender la ventaja clave de FastText para manejar palabras fuera de vocabulario (OOV - Out Of Vocabulary).
*   Cargar y usar vectores FastText pre-entrenados.
*   Comparar resultados y capacidades (especialmente OOV) con Word2Vec.
*   Reflexionar sobre cómo evaluar la calidad de los embeddings y detectar sesgos.

**Agenda:**
1.  Instalaciones e Importaciones
2.  Repaso Rápido: Word2Vec y sus limitaciones (OOV)
3.  GloVe: Vectores Globales desde Co-ocurrencias
4.  FastText: El Poder de las Subpalabras (¡Adiós OOV!)
5.  Cargando Vectores FastText Pre-entrenados
6.  Explorando FastText: Similitud, Analogías y ¡OOV!
7.  Comparativa: Word2Vec vs FastText (foco en OOV)
8.  Micro-Laboratorio (Ejercicio Práctico)
9.  Brainstorming: Evaluación y Detección de Sesgos

# 1. Instalaciones e Importaciones

In [1]:
# Necesitamos gensim principalmente
!pip install gensim > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [5]:
!pip uninstall gensim -y # Remove the existing gensim installation
!pip install gensim # Reinstall gensim to align with the NumPy version
# Restart the kernel to ensure the changes take effect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

In [1]:
import gensim
from gensim.models import KeyedVectors, FastText # Ahora importamos FastText también
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas.")

# **Importante:** Para este cuaderno, idealmente necesitaremos:
# 1. El modelo Word2Vec cargado previamente (para comparar).
# 2. Un modelo FastText pre-entrenado para español (¡necesitamos descargarlo!).

Librerías importadas.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path_to_fasttext = '/content/drive/MyDrive/glove_fasttext/wiki.es.bin'

In [3]:
# Intentar cargar el modelo FastText
try:
    print("Cargando vectores FastText (.bin)... (¡Esto puede tardar MUCHO tiempo y consumir RAM!)")
    # Usamos FastText.load_fasttext_format para cargar modelos .bin de FastText
    fasttext_model = gensim.models.fasttext.load_facebook_model(path_to_fasttext)
    # Los vectores están dentro del atributo .wv (como en Word2Vec cargado con KeyedVectors)
    fasttext_vectors = fasttext_model.wv
    print(f"¡Vectores FastText cargados! Vocabulario (estimado): {len(fasttext_vectors.index_to_key)} palabras. Dimensión: {fasttext_vectors.vector_size}")
    # Nota: El tamaño del vocabulario explícito puede ser menor en .bin, pero puede generar para OOV.
except FileNotFoundError:
    print(f"Error: No se encontró el archivo FastText en la ruta '{path_to_fasttext}'.")
    print("Por favor, descarga el archivo .bin pre-entrenado para español desde el sitio de FastText")
    print("y asegúrate de que la variable 'path_to_fasttext' tenga la ruta correcta.")
    fasttext_vectors = None
except Exception as e:
    print(f"Ocurrió un error al cargar el modelo FastText: {e}")
    fasttext_vectors = None

Cargando vectores FastText (.bin)... (¡Esto puede tardar MUCHO tiempo y consumir RAM!)
¡Vectores FastText cargados! Vocabulario (estimado): 985667 palabras. Dimensión: 300


In [4]:
path_to_word2vec = '/content/drive/MyDrive/glove_fasttext/SBW-vectors-300-min5.bin.gz' # La ruta del martes
word2vec_vectors = None

In [5]:
try:
    print("\nRecargando vectores Word2Vec...")
    word2vec_vectors = KeyedVectors.load_word2vec_format(path_to_word2vec, binary=True)
    print("Vectores Word2Vec recargados.")
except Exception as e:
    print(f"No se pudo recargar Word2Vec desde '{path_to_word2vec}': {e}")


Recargando vectores Word2Vec del martes...
Vectores Word2Vec recargados.


# 8. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:** (Asumiendo que `fasttext_vectors` y `word2vec_vectors` están cargados)

1.  **Comparación de Resultados:**
    *   Elegir 3 palabras que **sí** estén en ambos vocabularios (ej: 'gato', 'correr', 'inteligencia').
    *   Para cada palabra, obtener las 5 más similares usando `word2vec_vectors.most_similar()` y `fasttext_vectors.most_similar()`.
    *   Comparar las listas de similares. ¿Son idénticas? ¿Muy parecidas? ¿Diferentes? ¿Cuál les parece "mejor" o más coherente? Anotar observaciones.

2.  **Test OOV Exhaustivo:**
    *   Crear una lista propia de 10 palabras OOV. Incluyan:
        *   Errores tipográficos comunes (ej: "hobmre", "qeu", "dicimbre").
        *   Diminutivos/Aumentativos (ej: "perrito", "casita", "libraco").
        *   Formas verbales conjugadas (ej: "habíamos comido", "cantasteis").
        *   Palabras inventadas pero plausibles (ej: "tecnoestrés", "computofilia").
    *   Para **cada** palabra OOV de su lista:
        *   Verificar si da `KeyError` en `word2vec_vectors`.
        *   Obtener las 3 palabras más similares usando `fasttext_vectors`. Anotar los resultados. ¿Los similares que da FastText tienen algún sentido basado en las partes de la palabra OOV?

3.  **Discusión:**
    *   ¿En qué tipo de aplicación real (ej: un chatbot de atención al cliente, un sistema de recomendación de noticias, un corrector ortográfico) creen que la capacidad OOV de FastText marcaría una diferencia significativa respecto a usar Word2Vec? ¿Por qué?

In [6]:
if word2vec_vectors and fasttext_vectors:
    print("--- Comparando palabras comunes en Word2Vec y FastText ---")

    # Palabras que esperamos estén en ambos vocabularios
    common_words = ["gato", "correr", "inteligencia"]

    for palabra in common_words:
        print(f"\n--- Palabra: '{palabra}' ---")

        # Obtener similares con Word2Vec
        try:
            similares_w2v = word2vec_vectors.most_similar(palabra, topn=5)
            print(f"  Similares (Word2Vec): {similares_w2v}")
        except KeyError:
            print(f"  Word2Vec: '{palabra}' no encontrada en el vocabulario.")
        except Exception as e:
            print(f"  ERROR inesperado con Word2Vec para '{palabra}': {e}")

        # Obtener similares con FastText
        try:
            similares_ft = fasttext_vectors.most_similar(palabra, topn=5)
            print(f"  Similares (FastText): {similares_ft}")
        except KeyError:
             # This should not happen if the word is in the explicit vocab,
             # but FastText can still generate a vector even if not explicit.
             print(f"  FastText: '{palabra}' not found (unexpected for common word).")
        except Exception as e:
            print(f"  ERROR inesperado con FastText para '{palabra}': {e}")

else:
    print("\nAmbos modelos (Word2Vec y FastText) deben estar cargados para realizar la comparación.")

--- Comparando palabras comunes en Word2Vec y FastText ---

--- Palabra: 'gato' ---
  Similares (Word2Vec): [('perro', 0.8182171583175659), ('zorro', 0.7960419654846191), ('oso', 0.7677274346351624), ('conejo', 0.7547112703323364), ('mono', 0.7344450950622559)]
  Similares (FastText): [('gatos', 0.6868153214454651), ('conejo', 0.6618362665176392), ('perro', 0.6526236534118652), ('gatito', 0.6358351111412048), ('zorro', 0.6104276776313782)]

--- Palabra: 'correr' ---
  Similares (Word2Vec): [('corriendo', 0.6818697452545166), ('andar', 0.6599047183990479), ('saltar', 0.6438890099525452), ('caminar', 0.6328942179679871), ('corría', 0.5944115519523621)]
  Similares (FastText): [('correrse', 0.7749728560447693), ('corriendo', 0.7173823714256287), ('correrlo', 0.7168956398963928), ('correrla', 0.7070920467376709), ('correrlos', 0.6823992729187012)]

--- Palabra: 'inteligencia' ---
  Similares (Word2Vec): [('Inteligencia', 0.6916242837905884), ('contrainteligencia', 0.6242276430130005), ('HU

In [7]:
if word2vec_vectors and fasttext_vectors:
    print("\n--- Test OOV Exhaustivo ---")

    # Crear una lista propia de 10 palabras OOV
    my_oov_words = [
        "hobmre",         # Error tipográfico
        "qeu",            # Error tipográfico
        "dicimbre",       # Error tipográfico
        "perrito",        # Diminutivo
        "casota",         # Aumentativo (changed from casita to casota for variety)
        "libraco",        # Aumentativo
        "habíamoscomido", # Forma verbal conjugada (as one word)
        "cantasteis",     # Forma verbal conjugada
        "tecnoestrés",    # Palabra inventada/compuesta
        "computofilia"    # Palabra inventada
    ]

    for palabra in my_oov_words:
        print(f"\n--- Palabra OOV: '{palabra}' ---")

        # Intentar con Word2Vec
        print("  Intentando con Word2Vec:")
        try:
            vector_w2v = word2vec_vectors[palabra]
            similares_w2v = word2vec_vectors.most_similar(palabra, topn=3)
            print(f"    ¡Encontrada! Vector: {vector_w2v.shape}, Similares: {similares_w2v}")
        except KeyError:
            print("    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).")
        except Exception as e:
            print(f"    ERROR inesperado con Word2Vec: {e}")

        # Intentar con FastText
        print("  Intentando con FastText:")
        try:
            vector_ft = fasttext_vectors[palabra]
            similares_ft = fasttext_vectors.most_similar(palabra, topn=3)
            print(f"    ¡Vector generado! Vector: {vector_ft.shape}, Similares: {similares_ft}")
        except Exception as e:
            # No debería fallar por KeyError con un modelo .bin, pero podría haber otro error
            print(f"    ERROR inesperado con FastText: {e}")

else:
    print("\nAmbos modelos (Word2Vec y FastText) deben estar cargados para realizar el test OOV.")


--- Test OOV Exhaustivo ---

--- Palabra OOV: 'hobmre' ---
  Intentando con Word2Vec:
    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('nobmre', 0.6312950253486633), ('ademre', 0.5202481746673584), ('demre', 0.5149299502372742)]

--- Palabra OOV: 'qeu' ---
  Intentando con Word2Vec:
    ¡Encontrada! Vector: (300,), Similares: [('D-Mo', 0.6143811345100403), ('entenderias', 0.6070886850357056), ('encontrarias', 0.5987488031387329)]
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('qeuab', 0.6885186433792114), ('qeuad', 0.6670765280723572), ('wikisilki/pi', 0.6110560297966003)]

--- Palabra OOV: 'dicimbre' ---
  Intentando con Word2Vec:
    ¡Encontrada! Vector: (300,), Similares: [('Septiempre', 0.8246644139289856), ('Super_Jr._Tag_Tournament', 0.8202389478683472), ('noviebre', 0.8184508085250854)]
  Intentando con FastText:
    ¡Vector generado! Vector: (300,